<a href="https://colab.research.google.com/github/sgr1118/GD-NLP-Theory/blob/main/GD3_Vectorize_to_distribution_of_text(220926).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3-1. 들어가며

오늘은 단어의 빈도를 이용해서 텍스트를 표현하는 방법들과 텍스트의 분포를 이용해서 텍스트를 토큰화하는 방법에 대해서 배워보겠습니다. 오늘 다루게 되는 방법들은 Word2Vec 같은 임베딩을 활용한 분산 표현을 학습하는 기법이 대중화되기 이전에 많이 쓰이던 방법들입니다. 하지만 텍스트의 분포를 활용해서 텍스트를 벡터화하는 아이디어의 발전사를 통해 기초적인 개념을 잘 다져볼 수 있을 것입니다.

목차

1. 단어 빈도를 이용한 벡터화

(1) Bag of Words

(2) Bag of Words 구현해보기

(3) DTM과 코사인 유사도

(4) DTM의 구현과 한계점

(5) TF-IDF

(6) TF-IDF 구현하기

2. LSA와 LDA

(1) LSA

(2) LSA 실습

(3) LDA

(4) LDA 실습

3. 텍스트 분포를 이용한 비지도 학습 토크나이저

(1) 형태소 분석기와 단어 미등록 문제

(2) soynlp


# 3-2. 단어 빈도를 이용한 벡터화 (1) Bag of Words

벡터화가 필요한 이유는 머신러닝 모델은 텍스트보다는 수치화된 데이터를 더 잘 처리하기 때문이다.

벡터화 방법은 크게 (1) 통계와 머신 러닝을 활용한 방법, 그리고 (2) 인공 신경망을 활용하는 방법 두 가지로 나눠볼 수 있다.

1. Bag of Words (단어 가방)

- BOG는 자연어 처리 또는 정보 검색에서 쓰이는 간단한 단어 표현 방법이다. 문서 내의 단어들의 분포를 보고 이 문서의 특성을 파악하는 기법

- bog는 중복을 제거하지 않고 단어를 카운트하기 때문에 순서는 무시되고, 빈도 정보는 보존한다.

![](https://d3s0tskafalll9.cloudfront.net/media/images/GN-2-L-1.max-800x600.png)
<center></center>


In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2022-09-27 08:40:15--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2022-09-27 08:40:15--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce42057ef0a22afdd385c00a3b1.dl.dropboxusercontent.com/cd/0/get/BtvnjUHePqMZ0VtlGt_9rltglwXqB5rOBwkxZ2MAHWpN1jMj5DKHaeA44DECAyDA1Ir6-M4ItxPGH-1DFy8Ae_NCOn1rmDuT_v-JVdaThBCO04XIKqcczcFWbhETfLrD569RNFTzGoNnIwnHOtfUw3vUbVqutizLpB5naIrPE4Flwrv1rDMSsGwwcgBle750Dk0/file?dl=1# [following]
--2022-09-27 08:40:15--  https://uce42057ef0a22afdd385c00a3b1.dl.dropboxusercontent.com/cd/0/get/BtvnjUHePqMZ

In [ ]:
# bog 예시
from konlpy.tag import Mecab

mecab = Mecab()

def build_bag_of_words(document):
  # 온점 제거 및 형태소 분석
  document = document.replace('.', '')
  tokenized_document = mecab.morphs(document)

  word_to_index = {}
  bow = []

  for word in tokenized_document:  
    if word not in word_to_index.keys():
      word_to_index[word] = len(word_to_index)  
      # BoW에 전부 기본값 1을 넣는다.
      bow.insert(len(word_to_index) - 1, 1)
    else:
      # 재등장하는 단어의 인덱스
      index = word_to_index.get(word)
      # 재등장한 단어는 해당하는 인덱스의 위치에 1을 더한다.
      bow[index] = bow[index] + 1

  return word_to_index, bow

In [ ]:
doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."
vocab, bow = build_bag_of_words(doc1)
print('vocabulary :', vocab)
print('bag of words vector :', bow)

vocabulary : {'정부': 0, '가': 1, '발표': 2, '하': 3, '는': 4, '물가': 5, '상': 6, '승률': 7, '과': 8, '소비자': 9, '느끼': 10, '상승': 11, '률': 12, '은': 13, '다르': 14, '다': 15}
bag of words vector : [1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


# 3-3. 단어 빈도를 이용한 벡터화 (2) Bag of Words 구현해보기

tensorflow keras와 sklearn 라이브러리를 상여하여 bog 구현

1. keras Tokenizer 활용


케라스 토크나이저를 통해 Bag of Words를 만들어 보겠습니다.

In [ ]:
# keras Tokenizer

from tensorflow.keras.preprocessing.text import Tokenizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentence) # 단어장 생성
bow = dict(tokenizer.word_counts) # 각 단어와 단어의 빈도를 저장

print("Bag of Words :", bow) # bow 출력
print('단어장(Vocabulary)의 크기 :', len(tokenizer.word_counts)) # 중복을 제거한 단어들의 개수

Bag of Words : {'john': 1, 'likes': 3, 'to': 2, 'watch': 2, 'movies': 2, 'mary': 2, 'too': 1, 'also': 1, 'football': 1, 'games': 1}
단어장(Vocabulary)의 크기 : 10


여기서 단어장(Vocabulary) 이라는 또 다른 개념을 이해해 보겠습니다. 문헌에 따라 사전 또는 단어 집합이라고 불리는데요. 단어장이란 중복을 제거한 단어들의 집합을 말합니다. 이는 Bag of Words랑은 다른 개념입니다.

2. scikit-learn CountVectorizer 활용

Bag of Words를 만드는 또 다른 방법은 사이킷런의 CountVectorizer를 사용하는 것입니다.

In [ ]:
# scikit-learn CountVectorizer

from sklearn.feature_extraction.text import CountVectorizer

sentence = ["John likes to watch movies. Mary likes movies too! Mary also likes to watch football games."]

vector = CountVectorizer()
bow = vector.fit_transform(sentence).toarray()

print('Bag of Words : ', bow) # 코퍼스로부터 각 단어의 빈도수를 기록한다.
print('각 단어의 인덱스 :', vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

Bag of Words :  [[1 1 1 1 3 2 2 2 1 2]]
각 단어의 인덱스 : {'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


CounterVectorizer의 vector.fit_transform(입력 문장).toarray()으로 출력되는 Bag of Words 결과를 보면 각 단어의 빈도만 출력될 뿐, 어떤 단어의 빈도인지는 나오지 않습니다. [[1 1 1 1 3 2 2 2 1 2]]이라는 값만으로는 각 숫자가 어떤 단어의 빈도인지 알 수가 없지요?

이는 vector.vocabulary_를 통해서 각 단어에 부여된 인덱스를 확인하면 되는데요. 주의할 점은 이 인덱스는 0부터 시작한다는 점입니다. 가령, also의 인덱스는 0이네요. 그런데 BoW에서 0번째 위치한 값은 1이니까, also의 빈도는 1입니다. 또는 likes의 인덱스는 4인데, BoW에서 4번째 위치한 값은 3이니까 likes의 빈도는 3임을 알 수 있지요!

단어장이라는 개념은 Bag of Words를 위해 사용되는 개념이라기보다는, 자연어 처리에서 보편적으로 쓰이는 용어입니다. 그런데 이를 계속 강조하는 이유는 지금부터 사용할 DTM(Document-Term Matrix) 이라는 개념 이해를 돕기 위해서입니다. 


# 3-4. 단어 빈도를 이용한 벡터화 (3) DTM과 코사인 유사도

DTM(Document-Term Matrix) 은 직역하면 문서-단어 행렬입니다.
 
dtm은 여러 문서의 bog를 하나의 행렬로 구현한 것이다.

즉 dtm은 각 문서에 등장한 단어의 빈도수를 하나의 행렬로 통합시킨다.

dtm은 문서를 행, 단어는 열로 가지는 행렬이지만 그 반대로 하는경우 tdm이라고 할 수 있다.

예시

Doc 1: Intelligent applications creates intelligent business processes
Doc 2: Bots are intelligent applications
Doc 3: I do business intelligence

![](https://d3s0tskafalll9.cloudfront.net/media/images/GN-2-L-2.max-800x600.png)
<center></center>

점점 문자이 길어지다보면 단어 벡터 대부분의 값이 0이 되는 성질을 가지고있다.

이제 이 dtm을 사용하면 문서 간 코사인 유사도를 구할 수 있다.

문서1 : I like dog
문서2 : I like cat
문서3 : I like cat I like cat

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/GN-2-L-3.png)
<center></center>





In [ ]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

doc1 = np.array([0,1,1,1]) # 문서1 벡터
doc2 = np.array([1,0,1,1]) # 문서2 벡터
doc3 = np.array([2,0,2,2]) # 문서3 벡터

def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

Q1. 위 글을 읽고 문서1과 문서2 문서3의 관계 중 코사인 유사도가 1이 나올 관계를 예측해보세요.

문서2와 문서3의 관계가 1이 나올것이다.

두 벡터의 방향이 동일하기 때문이다.

다시 말하면 문서3은 문서2에서 단지 모든 단어의 빈도수가 1씩 증가할 뿐이다.

In [ ]:
print(cos_sim(doc1, doc2)) #문서1과 문서2의 코사인 유사도
print(cos_sim(doc1, doc3)) #문서1과 문서3의 코사인 유사도
print(cos_sim(doc2, doc3)) #문서2과 문서3의 코사인 유사도

0.6666666666666667
0.6666666666666667
1.0000000000000002


# 3-5. 단어 빈도를 이용한 벡터화 (4) DTM의 구현과 한계점

scikit-learn CountVectorizer 활용

사이킷런의 CountVectorizer를 이용하여 DTM을 만드는 방법은 앞서 Bag of Words를 만드는 방법과 동일합니다. 단지, 다수의 문서를 입력값으로 주면 됩니다. 다음은 3개의 문서에 대해서 DTM을 만드는 과정을 보여줍니다.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'John likes to watch movies',
    'Mary likes movies too',
    'Mary also likes to watch football games']
vector = CountVectorizer()

print(vector.fit_transform(corpus).toarray()) # 코퍼스로부터 각 단어의 빈도수를 기록.
print(vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

[[0 0 0 1 1 0 1 1 0 1]
 [0 0 0 0 1 1 1 0 1 0]
 [1 1 1 0 1 1 0 1 0 1]]
{'john': 3, 'likes': 4, 'to': 7, 'watch': 9, 'movies': 6, 'mary': 5, 'too': 8, 'also': 0, 'football': 1, 'games': 2}


1. DTM의 한계점

dtm은 bog 기바능로 문서를 비교하는 행렬이다. 그런데 dtm은 두 가지 한계점을 가진다.

(1) 문서와 단어의 수가 증가할수록 대부분의 행과 열은 0값을 가지게된다.

(2) 단어의 빈도에만 집중하는 방법 자체의 한계이다. 예를 들어 dtm을 만들 때, 불용어 'the'는 어떤 문서라던지 자주 등장하는 경향이 있습니다. 그런데 문서 1과 2에서 둘 다 'the'가 많이 등장한다고 두 문서는 유서하다고 할 수 없기 때문이다.


# 3-6. 단어 빈도를 이용한 벡터화 (5) TF-IDF

1. TF-IDF란

각 단어의 중요도를 판단하는 가중치를 주는 방법인 TF-IDF라는 방법이 있다.

TF-IDF는 모든 문서에서 자주 등장하는 단어는 중요도가 낮다고 판단하며, 특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단하는 것입니다.

TF-IDF는 불용어처럼 중요도가 낮으면서 모든 문서에 등장하는 단어들이 노이즈가 되는 것을 완화해 줍니다. 주의할 점은 TF-IDF를 사용하는 것이 DTM을 사용하는 것보다 성능이 항상 뛰어나지는 않다는 점입니다.

TF-IDF를 사용하기 위해서는 우선 DTM을 만든 뒤에 TF-IDF 가중치를 DTM에 적용합니다.


2. TF-IDF 계산하기

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/GN-2-L-4.png)
<center>TF-IDF 수식</center>

y = 문서, x = 단어라고 할 때 TF-IDF의 수식을 보여줍니다.

tf뒤에 곱해지는 log항이 idf이다. idf를 구하는 방법은 아래 예시로 이해해본다.

- Question. 전체 문서의 수가 5개라고 해봅시다. 그리고 단어 'like'가 문서2에서 200번, 문서 3에서 300번 등장했다고 해봅시다. 다른 문서에서 단어 'like'는 등장하지 않았습니다. 이때, 단어 'like'의 IDF는 몇일까요?

df = 2, 문서2, 3에서 등장했기 때문이다.

N = 5, 전체 문서의 개수

idf = ln5/2 = 0.91629073187

- Question. 문서2과 문서3에서의 단어 'like'의 TF-IDF값은 몇 일까요?

문서 2의 TF-IDF : 200 * ln5/2 = 183.258146375

문서 3의 TF-IDF : 300 * ln5/2 = 274.887219562

문서3의 TF-IDF값이 더 큰 이유는 특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단하는 TF-IDF의 목적에 알맞다.

[TF-IDF 설명 영상](https://youtu.be/Rd3OnBPDRbM)


# 3-7. 단어 빈도를 이용한 벡터화 (6) TF-IDF 구현하기

1. TF-IDF 구현하기

TF-IDF를 실제로 파이썬을 통해 구현해보겠습니다! 우선 필요한 도구를 임포트합니다.

In [ ]:
# 필요한 라이브러리 불러오기

from math import log
import pandas as pd

In [ ]:
docs = ['John likes to watch movies and Mary likes movies too',
  'James likes to watch TV',
  'Mary also likes to watch football games']

In [ ]:
# dtm의 열을 만들기 위해서 문서 3개의 단어들이 모두 들어간 통합 단어장 만들기

vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()
print('단어장의 크기 :', len(vocab))
print(vocab)

# 중복을 제외한 단어의 개수는 13개

# 총 문서의 수를 N에 저장

N = len(docs)

단어장의 크기 : 13
['James', 'John', 'Mary', 'TV', 'also', 'and', 'football', 'games', 'likes', 'movies', 'to', 'too', 'watch']


이제 TF 함수, IDF 함수, TF-IDF 함수를 만들겠습니다.

여기서 IDF를 실제 구현할 때는 앞서 배운 식과는 다소 다른 식을 사용하게 됩니다. 실제로 많은 파이썬 패키지들이 앞에서 배운 식에서 조금씩 조정된, 서로 다른 식을 사용합니다.

우선, 여기서는 log 항의 분모에 1을 더해주겠습니다. 이는 (Bag of Words를 사용할 때 일반적인 상황은 아니지만) 특정 단어가 전체 문서에서 등장하지 않을 경우에 분모가 0이 되는 상황을 방지하기 위함입니다. 그리고 log 항에 1을 더해주겠습니다. 이는 log 항의 분자와 분모값이 동일해졌을 때, log의 진수가 1이 됨에 따라서 IDF의 값이 0이 되는 것을 방지하기 위함입니다.

In [ ]:
def tf(t, d):
  return d.count(t)

def idf(t):
  df = 0
  for doc in docs:
    df += t in doc
  return log(N/(df+1)) +1

def tfidf(t,d):
  return tf(t, d) * idf(t)

In [ ]:
# tf함수를 사용하여 DTM만들기

result = []

for i in range(N): # 각 문서에 대해서 아래 명령을 수행
  result.append([])
  d = docs[i]
  for j in range(len(vocab)):
    t = vocab[j]

    result[-1].append(tf(t,d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0,1,1,0,0,1,0,0,2,2,2,1,1
1,1,0,0,1,0,0,0,0,1,0,1,0,1
2,0,0,1,0,1,0,1,1,1,0,1,0,1


In [ ]:
# IDF만들기

result = []
for j in range(len(vocab)):
  t = vocab[j]
  result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns = ['IDF'])
idf_

,IDF
James,1.405465
John,1.405465
Mary,1.000000
TV,1.405465
also,1.405465
and,1.405465
football,1.405465
games,1.405465
likes,0.712318
movies,1.405465


In [ ]:
idf_.sort_values(by = 'IDF', ascending='False')

,IDF
likes,0.712318
to,0.712318
watch,0.712318
Mary,1.000000
James,1.405465
John,1.405465
TV,1.405465
also,1.405465
and,1.405465
football,1.405465


In [ ]:
# TF-IDF

result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]
        
        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,James,John,Mary,TV,also,and,football,games,likes,movies,to,too,watch
0,0.000000,1.405465,1.0,0.000000,0.000000,1.405465,0.000000,0.000000,1.424636,2.81093,1.424636,1.405465,0.712318
1,1.405465,0.000000,0.0,1.405465,0.000000,0.000000,0.000000,0.000000,0.712318,0.00000,0.712318,0.000000,0.712318
2,0.000000,0.000000,1.0,0.000000,1.405465,0.000000,1.405465,1.405465,0.712318,0.00000,0.712318,0.000000,0.712318


2. scikit-learn TFidVectorizer 활용

skelanr에서 TF-IDF를 자동으로 계산하여 출력하는 TfidfVectorizer를 제공합니다.

향후 실습 중에 결괏값이 예상과 달라 혼란이 일어나는 일이 없도록 미리 언급하자면, 사이킷런의 TfidfVectorizer 또한 이번에 배운 기본식에서 조정된 식을 사용합니다. 간단히 언급하면, TfidfVectorizer는 위에서 우리가 파이썬으로 구현한 식에서 더 나아가 loglog 항의 분자에도 1을 더해주며, TF-IDF의 결과에 L2 Norm까지 추가로 수행한다는 점이 다릅니다.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
  'John likes to watch movies and Mary likes movies too',
  'James likes to watch TV',
  'Mary also likes to watch football games',  
]

tfidfv = TfidfVectorizer().fit(corpus)
vocab = list(tfidfv.vocabulary_.keys()) # 단어장을 리스트로 저장
vocab.sort() # 단어장을 알파벳 순으로 정렬

# TF-IDF 행렬에 단어장을 데이터프레임의 열로 지정하여 데이터프레임 생성
tfidf_ = pd.DataFrame(tfidfv.transform(corpus).toarray(), columns = vocab)
tfidf_

,also,and,football,games,james,john,likes,mary,movies,to,too,tv,watch
0,0.000000,0.321556,0.000000,0.000000,0.000000,0.321556,0.379832,0.244551,0.643111,0.189916,0.321556,0.000000,0.189916
1,0.000000,0.000000,0.000000,0.000000,0.572929,0.000000,0.338381,0.000000,0.000000,0.338381,0.000000,0.572929,0.338381
2,0.464997,0.000000,0.464997,0.464997,0.000000,0.000000,0.274634,0.353642,0.000000,0.274634,0.000000,0.000000,0.274634


# 3-8. LSA와 LDA (1) LSA

특정 단어(단어의 의미)와 관련된 문서 집합(문서의 주제)을 찾아내고 싶다면 어떨까요? 단어들의 빈도가 아닌 의미와 주제를 알고 싶다면 어떻게 해야 할까요? 이는 DTM이나 TF-IDF 만으로는 풀기 어려운 문제입니다. 이를 해결할 수 있는 LSA(Latent Semantic Analysis) 에 대해 알아봅시다.

1. LSA(Latent Semantic Analysis)

- 전체 코퍼스에서 문서 속 단어들 사이의 관계를 찾아내는 자연어 처리 정보 검색 기술입니다.

- LSA를 사용하면 단어와 단어 사이, 문서와 문서 사이, 단어와 문서 사이의 의미적 유사성 점수를 찾아낼 수 있습니다. 그리고 이 방법은 어떤 문서에서 특정 단어들의 빈도가 몇인지를 판단하는 것보다 효과적인 경우가 많습니다.

2. SVD(Singular Value Decompotion)
- LSA를 이해하기 위해서 SVD를 이해할 필요가 있다. 아래 참고 자료를 확인하길 권장
[머신러닝 - 19. 고유값(eigenvalue), 고유벡터(eigenvector), 고유값 분해(eigen decomposition)](https://bkshin.tistory.com/entry/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D-19-%ED%96%89%EB%A0%AC?category=1057680)

Q2. 단위 행렬의 정의는 무엇인가요?
- 대각 성분이 모두 1이며 나머지 성분은 모두 0인 정사각행렬

Q3. n차 정사각행렬 행렬 A에 대해 어떤 행렬을 곱했을 때, 결과 행렬이 단위 행렬이라면 이 어떤 행렬을 A의 무슨 행렬이라고 하나요?
-역행렬

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/GN-2-L-9.png)
<center></center>

특잇값 분해(Singular Value Decomposition, SVD) 란 m × n 크기의 임의의 사각 행렬 A를 위의 그림의 Full SVD와 같이 특이 벡터(singular vector)의 행렬와 특잇값(singular value)의 대각행렬로 분해하는 것을 말합니다. 특잇값 분해를 좀 더 자세히 이해하기 위해서 아래의 글을 도입부부터 '파이썬을 사용한 특이 분해'까지만 읽고 오겠습니다. 좀더 직관적이고 쉬운 설명이 필요하다면 그 아래의 동영상 링크를 함께 참고해 주세요.

[데이터 사이언스 스쿨 : SVD](https://datascienceschool.net/02%20mathematics/03.04%20%ED%8A%B9%EC%9E%87%EA%B0%92%20%EB%B6%84%ED%95%B4.html)

[SVD의 목적](https://www.youtube.com/watch?v=vxJ1MzfvL5w)

Q4. 행렬 A에 특잇값 분해를 수행하면 U, Σ, V라는 세 가지 행렬로 분해됩니다. 이때, 특잇값(singular value)은 어디서 얻을 수 있나요?
- Σ의 대각 성분

특잇값 분해 중에서 조금 특별한 특잇값 분해가 있습니다. 특잇값 가운데 가장 큰(다시 말해, 가장 중요한) t개만 남기고 해당 특잇값에 대응되는 특이 벡터(singular vector) 들로 행렬 A를 근사(approximate) 하도록 하면, 이를 절단된 특잇값 분해(Truncated SVD) 라고 합니다.

Truncated SVD를 수행하면 행렬 Σ의 대각 원솟값 중에서 상윗값 t개만 남게 되며, U행렬과 V행렬의 t열까지만 남습니다. 이로 인해 세 행렬에서 값(정보)의 손실이 일어나 기존의 행렬 A를 정확히 복구할 수는 없게 됩니다.

여기서 t는 하이퍼파라미터입니다. 하이퍼파라미터란 사용자가 직접 값을 선택하며 성능에 영향을 주는 매개변수를 말합니다. t를 크게 잡으면 기존의 행렬 A로부터 다양한 의미를 가져갈 수 있지만, 노이즈를 제거하려면 t를 작게 잡아야 합니다.



2. LSA와 Truncated SVD

![](https://d3s0tskafalll9.cloudfront.net/media/images/GN-2-L-10.max-800x600.png)
<center></center>

A=USV^T




# 3-9. LSA와 LDA (2) LSA 실습

In [ ]:
# 필요한 라이브러리 불러오기

import pandas as pd
import numpy as np
import urllib.request
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# 데이터 다운로드

urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", 
                           filename='abcnews-date-text.csv')

('abcnews-date-text.csv', <http.client.HTTPMessage at 0x7ff04128e250>)

In [ ]:
# 데이터 불러오기
data = pd.read_csv('/content/abcnews-date-text.csv', error_bad_lines=False)
data.shape

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


(1082168, 2)

In [ ]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


In [ ]:
text = data[['headline_text']].copy()
text.head()

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


In [ ]:
text.nunique()
# 중복을 제외하고 유일한 시퀀스를 가지는 샘플의 개수를 출력

headline_text    1054983
dtype: int64

In [ ]:
text.drop_duplicates(inplace=True) # 중복 샘플 제거
text.reset_index(drop=True, inplace=True)
text.shape

(1054983, 1)

1. 데이터 정제 및 정규화

이제 텍스트 데이터를 정제 및 정규화하는 과정을 진행해 보겠습니다. 우선 NLTK의 토크나이저를 이용해 전체 텍스트 데이터에 대해서 단어 토큰화를 수행하고, NLTK가 제공하는 불용어 리스트를 사용하여 불용어를 제거합니다.

In [ ]:
# NLTK 토크나이저를 이용해서 토큰화

text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(
    row['headline_text']), axis = 1)

# 불용어 제거
stop_words = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x:
                                                    [word for word in x if word not in (stop_words)])
text.head()

,headline_text
0,"[aba, decides, community, broadcasting, licence]"
1,"[act, fire, witnesses, must, aware, defamation]"
2,"[g, calls, infrastructure, protection, summit]"
3,"[air, nz, staff, aust, strike, pay, rise]"
4,"[air, nz, strike, affect, australian, travellers]"


In [ ]:
# 단어 정규화. 3인칭 단수 표현 -> 1인칭 변환, 과거형 동사 -> 현재형 동사 등을 수행한다.
nltk.download('omw-1.4')

text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

# 길이가 1 ~ 2인 단어는 제거.
text = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 2])
print(text[:5])

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


0     [aba, decide, community, broadcast, licence]
1    [act, fire, witness, must, aware, defamation]
2       [call, infrastructure, protection, summit]
3            [air, staff, aust, strike, pay, rise]
4    [air, strike, affect, australian, travellers]
Name: headline_text, dtype: object


2. 역토큰화 및 DTM 생성

DTM을 생성하는 CountVectorizer 또는 TF-IDF 행렬을 생성하는 TfidfVectorizer의 입력으로 사용하기 위해서 토큰화 과정을 역으로 되돌리는 역토큰화(detokenization) 를 수행해보겠습니다.

In [ ]:
# 역토큰화 (토큰화 작업을 역으로 수행)

detokenized_doc = []
for i in range(len(text)):
  t = ' '.join(text[i])
  detokenized_doc.append(t)

train_data = detokenized_doc

In [ ]:
train_data[:5]

['aba decide community broadcast licence',
 'act fire witness must aware defamation',
 'call infrastructure protection summit',
 'air staff aust strike pay rise',
 'air strike affect australian travellers']

전처리 최종 결과인 train_data는 다음에 배울 LDA 실습에서도 재사용 할 예정입니다. CountVectorizer를 사용하여 DTM을 생성해봅시다. 단어의 수는 5,000개로 제한하겠습니다.

In [ ]:
# 상위 5000개의 단어만 사용

c_vectorizer = CountVectorizer(stop_words='english', max_features = 5000)
document_term_matrix = c_vectorizer.fit_transform(train_data)

In [ ]:
# DTM 크기 확인
print('행렬의 크기 :',document_term_matrix.shape)

# DTM의 크기는 (문서의 수 * 단어 집합의 크기)

행렬의 크기 : (1054983, 5000)


3. scikit-learn TruncatedSVD 활용

이제 Truncated SVD를 통해 LSA를 수행해 봅시다. 토픽의 수를 10으로 정하겠습니다. 이는 앞서 배운 하이퍼파라미터 kk에 해당되며, 행렬 V_k^T가 k × (단어의 수)의 크기를 가지도록 DTM에 TruncatedSVD를 수행합니다.

[sklearn.decomposition.TruncatedSVD.fit_transform()](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html#sklearn.decomposition.TruncatedSVD.fit_transform)

In [ ]:
from sklearn.decomposition import TruncatedSVD

n_topics = 10
lsa_model = TruncatedSVD(n_components = n_topics)
lsa_model.fit_transform(document_term_matrix)

array([[ 1.20447841e-02, -3.69668322e-03,  1.82902635e-02, ...,
         6.06885055e-03,  3.38835211e-03,  5.83465090e-03],
       [ 2.90591344e-02, -1.08596859e-02,  1.82761612e-02, ...,
         8.69601155e-04, -1.02225391e-02, -1.50662024e-03],
       [ 5.03173835e-03, -2.01418272e-03,  9.76770070e-03, ...,
        -2.17010892e-03,  1.84423786e-03,  8.76599945e-04],
       ...,
       [ 2.96831844e-02,  4.30630549e-03,  2.48735192e-02, ...,
         3.42735880e-02,  9.24561961e-03,  3.16049833e-02],
       [ 6.16861584e-02, -5.62745631e-03,  1.35365953e-01, ...,
         8.01687216e-01,  7.87048650e-01, -3.79692970e-01],
       [ 7.15639955e-02,  2.87330452e-02,  6.80416575e-05, ...,
         1.38029307e-02, -1.58313165e-03,  5.97453295e-02]])

In [ ]:
# TruncatedSVD를 통해 얻은 행렬 V_k^T의 크기를 확인해봅시다.

print(lsa_model.components_.shape)

(10, 5000)


행렬 V_k^T 가 k × (단어의 수) 의 크기를 가지는 것을 확인할 수 있습니다. 이제 각 행을 전체 코퍼스의 kk개의 주제(topic)로 판단하고 각 주제에서 n개씩 단어를 출력해 봅시다.

In [ ]:
terms = c_vectorizer.get_feature_names() # 단어 집합 5000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
  for idx, topic in enumerate(components):
    print('Topic %d:' % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lsa_model.components_, terms)

Topic 1: [('police', 0.74634), ('man', 0.45354), ('charge', 0.2108), ('new', 0.14091), ('court', 0.11147)]
Topic 2: [('man', 0.69432), ('charge', 0.30048), ('court', 0.168), ('face', 0.11451), ('murder', 0.10561)]
Topic 3: [('new', 0.83661), ('plan', 0.23617), ('say', 0.18266), ('govt', 0.11033), ('council', 0.1101)]
Topic 4: [('say', 0.73911), ('plan', 0.35839), ('govt', 0.16775), ('council', 0.12714), ('urge', 0.07531)]
Topic 5: [('plan', 0.7337), ('council', 0.17407), ('govt', 0.14112), ('urge', 0.08251), ('water', 0.06713)]
Topic 6: [('govt', 0.53411), ('court', 0.26839), ('urge', 0.24413), ('fund', 0.2085), ('face', 0.16832)]
Topic 7: [('charge', 0.50867), ('court', 0.45801), ('face', 0.3523), ('plan', 0.11784), ('murder', 0.11294)]
Topic 8: [('win', 0.57527), ('court', 0.34553), ('kill', 0.22444), ('crash', 0.16058), ('council', 0.11201)]
Topic 9: [('win', 0.58933), ('charge', 0.48868), ('council', 0.16216), ('cup', 0.06519), ('world', 0.05689)]
Topic 10: [('kill', 0.59598), ('cr

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# 3-10. LSA와 LDA (3) LDA

앞서 LSA를 통해 전체 코퍼스로부터 주요 주제인 토픽을 찾아내는 실습을 해봤습니다. 이처럼 문서의 집합에서 토픽을 찾아내는 프로세스를 토픽 모델링(Topic Modelling) 이라고 합니다. 이는 고객의 소리와 같이 많은 문서에서 주요 주제를 알아내는 일이 중요할 때 사용합니다.

이제부터 배울 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA) 은 토픽 모델링의 또 다른 대표적인 알고리즘입니다. 간단하게 얘기하자면, LDA는 문서들이 토픽들의 혼합으로 구성되어 있으며, 토픽들은 확률 분포에 기반하여 단어들을 생성한다고 가정합니다. 그리고 데이터가 주어지면, LDA는 이 가정에 따라 단어들의 분포로부터 문서가 생성되는 과정을 역추적해 문서의 토픽을 찾아냅니다.

1. LDA(Latent Dirichlet Allocation) 시뮬레이션

[LDA Topic Modeling Simulation](https://lettier.com/projects/lda-topic-modeling/)

위의 사이트는 별도의 코드 작성 없이 입력한 문서들로부터 DTM을 만들고 LDA를 수행한 결과를 보여주는 웹 사이트입니다. 위 사이트에 접속하여 다음과 같은 순서로 시뮬레이션을 진행해 주세요.

'+ Add Document'를 통해 다수의 단어로 구성된 다수의 문서를 추가하세요.

좌측의 'Topics'의 값을 원하는 값으로 지정해보세요. 이 값은 이 코퍼스에서 몇 개의 주제를 얻을 것인지 정하는 하이퍼파라미터값입니다.

'Run LDA'를 누르고 결과로 나오는 두 개의 행렬을 확인합니다.

Q5. 위 사이트로 LDA를 시뮬레이션하면 두 개의 행렬이 결과로 나옵니다. 각각의 행렬에서 행과 열은 무엇으로 구성되어 있나요?
- 첫 번째 행렬 : 첫 번째 행렬의 행은 단어 집합의 단어들이고, 열은 Topic입니다.
- 두 번째 행렬 : 두 번째 행렬의 행은 문서이고, 열은 Topic입니다.


2. LDA(Latent Dirichlet Allocation)의 두 가지 결과

위의 시뮬레이션을 통해 두 개의 행렬이 결과로서 나오는 것을 확인하셨나요? LDA는 각 토픽의 단어 분포와 각 문서의 토픽 분포를 추정해냅니다.

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/GN-2-L-13.png)
<center></center>




3. LDA의 가정

LDA는 전체 코퍼스, 즉 다수의 문서들로부터 토픽을 뽑아내기 위해서 하나의 가정을 염두에 두고 있습니다. 모든 문서 하나, 하나가 작성될 때 그 문서의 작성자는 아래와 같은 생각을 했다는 가정입니다.

- 나는 이 문서를 작성하기 위해서 이런 주제들을 넣을거고, 이런 주제들을 위해서는 이런 단어들을 넣을 거야.'

4. LDA의 직관 이해하기

[위키독스: 잠재 디리클레 할당](https://wikidocs.net/30708)

Q6. LSA와 LDA의 메커니즘을 각각 한 줄로 요약해서 설명해보세요.

(1) LSA : DTM을 차원 축소 하여 축소 차원에서 근접 단어들을 토픽으로 묶는다.

(2) LDA : 단어가 특정 토픽에 존재할 확률과 문서에 특정 토픽이 존재할 확률을 결합확률로 추정하여 토픽을 추출한다.

[LDA 모형 유튜브](https://serviceapi.nmv.naver.com/ugc/flash/convertIframeTag?vid=A008B9E1EAFC02C99F92928155487839090E&outKey=V1210ad4156cf64ce0c6a3e18cecaae499f6528784c999ca6541c3e18cecaae499f65&width=544&height=306)



# 3-11. LSA와 LDA (4) LDA 실습

LDA 실습은 LSA 실습 과정에서 만들어둔 전처리가 완료된 train_data를 재사용한다고 가정합니다. 해당 데이터는 역토큰화 과정을 거친 데이터입니다.



1. TF-IDF 행렬 생성

LDA는 DTM 또는 TF-IDF를 입력으로 받을 수 있습니다. 여기서는 TF-IDF를 사용하겠습니다.

TfidfVectorizer를 사용하여 TF-IDF 행렬을 생성해봅시다. 단어의 수는 5,000개로 제한하겠습니다. TF-IDF 행렬을 생성한 후에는 행렬의 크기를 확인합시다.

In [ ]:
# 상위 5000갸의 단어만 사용

tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tf_idf_matrix = tfidf_vectorizer.fit_transform(train_data)

# TF-IDF 행렬의 크기를 확인해봅시다.
print('행렬의 크기 :', tf_idf_matrix.shape)

행렬의 크기 : (1054983, 5000)


2. scikit-learn LDA Model 활용

사이킷런의 LDA 모델을 사용하여 학습합니다. LSA와 마찬가지로 동일한 사이킷런 패키지이므로 앞으로 진행되는 실습 과정은 LSA와 매우 유사합니다. 토픽의 개수는 10개로 정했습니다. 이는 n_components의 인자값입니다. 시간이 수분 정도 소요될 것이므로 느긋하게 기다려 주세요.

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components=10, learning_method='online', random_state=777, max_iter=1)
lda_model.fit_transform(tf_idf_matrix)

array([[0.0335099 , 0.0335099 , 0.0335099 , ..., 0.17024867, 0.0335099 ,
        0.0335099 ],
       [0.03365631, 0.03365631, 0.03365631, ..., 0.03365631, 0.03365631,
        0.03365631],
       [0.25184095, 0.0366096 , 0.0366096 , ..., 0.0366096 , 0.0366096 ,
        0.0366096 ],
       ...,
       [0.26687206, 0.02914502, 0.02914502, ..., 0.13007484, 0.02916018,
        0.28739608],
       [0.10378115, 0.02637829, 0.12325014, ..., 0.02637829, 0.02637829,
        0.02637829],
       [0.03376055, 0.03376055, 0.2255442 , ..., 0.03376055, 0.03376055,
        0.03376055]])

In [ ]:
print(lda_model.components_.shape)

(10, 5000)


In [ ]:
terms = tfidf_vectorizer.get_feature_names() # 단어 집합 5000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
  for idx, topic in enumerate(components):
    print('Topic %d:' % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lda_model.components_, terms)

Topic 1: [('australia', 9359.06334), ('sydney', 5854.97288), ('attack', 4784.76322), ('change', 4193.63035), ('year', 3924.88997)]
Topic 2: [('government', 6344.07413), ('charge', 5947.12292), ('man', 4519.7974), ('state', 3658.16422), ('live', 3625.10473)]
Topic 3: [('australian', 7666.65651), ('say', 7561.01807), ('police', 5513.22932), ('home', 4048.38409), ('report', 3796.04446)]
Topic 4: [('melbourne', 5298.35047), ('south', 4844.59835), ('death', 4281.78433), ('china', 3214.44581), ('women', 3029.28443)]
Topic 5: [('win', 5704.0914), ('canberra', 4322.0963), ('die', 4025.63057), ('open', 3771.65243), ('warn', 3577.47151)]
Topic 6: [('court', 5246.3124), ('world', 4536.86331), ('country', 4166.34794), ('woman', 3983.97748), ('crash', 3793.50267)]
Topic 7: [('election', 5418.5038), ('adelaide', 4864.95604), ('house', 4478.6135), ('school', 3966.82676), ('2016', 3955.11155)]
Topic 8: [('trump', 8189.58575), ('new', 6625.2724), ('north', 3705.40987), ('rural', 3521.42659), ('donald',

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


# 3-12. 텍스트 분포를 이용한 비지도 학습 토크나이저 (1) 형태소 분석기와 단어 미등록 문제

지금까지 띄어쓰기 단위의 단어 분포를 이용해서 단어나 문서를 벡터화하는 방법에 대해 알아봤습니다. 이번에는 텍스트의 분포를 이용해서 토큰화를 수행하는 비지도 학습 토크나이저에 대해서 알아봅시다.

1. 형태소 분석기의 필요성

한국어는 교착어입니다. 교착어란 하나의 낱말(엄밀히는 하나의 어절)이 하나의 어근(root)(혹은 어간(stem))과 각각 단일한 기능을 가지는 하나 이상의 접사(affix)의 결합으로 이루어져 있는 언어를 말합니다. 조사라는 품사는 교착어에만 존재합니다.

이런 특성으로 인해 한국어는 영어와 달리 조사나 접사가 존재하며, 영어처럼 띄어쓰기 단위 토큰화가 제대로 동작하지 않습니다.

영어 토큰화를 복습해봅시다. 다음과 같이 영어 문장은 단순히 띄어쓰기를 수행해도 단어 토큰화가 제대로 동작합니다.



In [ ]:
en_text = "The dog ran back to the corner near the spare bedrooms"
print(en_text.split())

['The', 'dog', 'ran', 'back', 'to', 'the', 'corner', 'near', 'the', 'spare', 'bedrooms']


In [ ]:
kor_text = "사과의 놀라운 효능이라는 글을 봤어. 그래서 오늘 사과를 먹으려고 했는데 사과가 썩어서 슈퍼에 가서 사과랑 오렌지 사 왔어"
print(kor_text.split())

['사과의', '놀라운', '효능이라는', '글을', '봤어.', '그래서', '오늘', '사과를', '먹으려고', '했는데', '사과가', '썩어서', '슈퍼에', '가서', '사과랑', '오렌지', '사', '왔어']


위의 예제 문장에서 '사과'란 단어가 총 4번 등장했는데 모두 '의', '를', '가', '랑' 등이 붙어있어 이를 제거해주지 않으면 기계는 전부 다른 단어로 인식하게 됩니다. 이를 해결하기 위해서 한국어는 주로 형태소 분석기를 사용합니다. 가령, 형태소 분석기 중 하나인 Okt를 사용해서 형태소 분석을 할 경우에는 다음 결과를 얻습니다.

In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.1 MB/s 
     |████████████████████████████████| 453 kB 43.4 MB/s 


In [ ]:
from konlpy.tag import Okt

tokenizer = Okt()
print(tokenizer.morphs(kor_text))

['사과', '의', '놀라운', '효능', '이라는', '글', '을', '봤어', '.', '그래서', '오늘', '사과', '를', '먹으려고', '했는데', '사과', '가', '썩어서', '슈퍼', '에', '가서', '사과', '랑', '오렌지', '사', '왔어']


2. 단어 미등록 문제

하지만 기존의 형태소 분석기는 등록된 단어를 기준으로 형태소를 분류해 내어 새롭게 만들어진 단어를 인식하기 어렵다는 특징이 있습니다.


In [ ]:
print(tokenizer.morphs('모두의연구소에서 자연어 처리를 공부하는 건 정말 즐거워'))

['모두', '의', '연구소', '에서', '자연어', '처리', '를', '공부', '하는', '건', '정말', '즐거워']


'모두의연구소'는 하나의 단어이지만 위의 형태소 분석 결과에서는 전부 분리된 결과를 보여줍니다.

그렇다면 텍스트 데이터에서 특정 문자 시퀀스가 함께 자주 등장하는 빈도가 높고, 앞뒤로 조사 또는 완전히 다른 단어가 등장하는 것을 고려해서 해당 문자 시퀀스를 형태소라고 판단하는 형태소 분석기라면 어떨까요?

예를 들어, '모두의연구소'라는 문자열이 자주 연결되어 등장한다면 형태소라고 판단하고, '모두의연구소'라는 단어 앞, 뒤에 '최고', 'AI', '실력'과 같은 독립된 다른 단어들이 계속해서 등장한다면 '모두의연구소'를 형태소로 파악하는 식이지요. 이런 아이디어를 가진 형태소 분석기가 soynlp입니다.

# 3-13. 텍스트 분포를 이용한 비지도 학습 토크나이저 (2) soynlp

soynlp는 품사 태깅, 형태소 분석 등을 지원하는 한국어 형태소 분석기입니다.
비지도 학습으로 형태소 분석을 한다는 특징을 갖고 있으며, 데이터에 자주 등장하는 단어들을 형태소로 분석합니다.
soynlp 형태소 분석기는 내부적으로 단어 점수표로 동작합니다. 이 점수는 응집 확률(cohesion probability) 과 브랜칭 엔트로피(branching entropy) 를 활용합니다.

1. suynlp 실습

In [ ]:
import urllib.request

urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt",\
                            filename='2016-10-20.txt')

('2016-10-20.txt', <http.client.HTTPMessage at 0x7f445de1ced0>)

In [ ]:
!pip install soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 416 kB 25.6 MB/s 


In [ ]:
# 말뭉치를 문서 단위로 분리

from soynlp import DoublespaceLineCorpus

# 말뭉치에 대해서 다수의 문서로 분리
corpus = DoublespaceLineCorpus('/content/2016-10-20.txt')
len(corpus)

30091

In [ ]:
# 공백이 아닌 문서에 한해 상위 3개의 문서만 출력
i = 0
for document in corpus:
  if len(document) > 0 :
    print(document)
    i = i + 1
  if i == 3:
    break

19  1990  52 1 22
오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에

In [ ]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

training was done. used memory 1.591 Gb


soynlp는 비지도학습 형태소 분석기이므로 기존의 형태소 분석기와는 달리 학습 과정을 거쳐야 합니다. 전체 코퍼스로부터 응집 확률과 브랜칭 엔트로피 단어 점수표를 만드는 과정이지요. WordExtractor.extract()를 통해서 전체 코퍼스에 대해 단어 점수표를 계산할 수 있습니다.

2. soynlp의 응집 확률(cohesion probability)

응집 확률은 내부 문자열(substring)이 얼마나 응집하여 자주 등장하는지를 판단하는 척도입니다. 응집 확률은 문자열을 문자 단위로 분리하여 내부 문자열을 만드는 과정에서, 왼쪽부터 순서대로 문자를 추가하면서 각 문자열이 주어졌을 때 그다음 문자가 나올 확률을 계산하여 누적 곱을 한 값입니다. 이 값이 높을수록 전체 코퍼스에서 이 문자열 시퀀스는 하나의 단어로 등장할 가능성이 높습니다. 수식은 아래와 같습니다.

![](https://d3s0tskafalll9.cloudfront.net/media/original_images/GN-2-L-14.png)
<center></center>

![](https://d3s0tskafalll9.cloudfront.net/media/images/GN-2-L-15.max-800x600.png)
<center></center>



In [ ]:
text = ['반포한', '반포한강', '반포한강공', '반포한강공원', '반포한강공원에']
result = []
for i in text:
  soy = word_score_table[i].cohesion_forward
  result.append(soy)

In [ ]:
result

[0.08838002913645132,
 0.19841268168224552,
 0.2972877884078849,
 0.37891487632839754,
 0.33492963377557666]

3. soynlp의 브랜칭 엔트로피(branching entropy)

- 브렌창 엔트로피는 확률 분포의 엔트로피값을 사용합니다. 이는 주어진 문자열에서 다음 문자가 등장할 수 있는 가능성을 판단하는 척도입니다.

- 



In [ ]:
# 브랜칭 엔트로피 실행해보기
text = ['디스','디스플', '디스플레', '디스플레이']
result = []
for i in text:
  soy = word_score_table[i].right_branching_entropy
  result.append(soy)

result

[1.6371694761537934, -0.0, -0.0, 3.1400392861792916]

'디스' 다음에는 다양한 문자가 올 수 있으니까 1.63이라는 값을 가지는 반면, '디스플'이라는 문자열 다음에는 다음 문자로 '레'가 오는 것이 너무나 명백하기 때문에 0이란 값을 가집니다.

'디스플레' 다음도 마찬가지겠죠. 다음 문자가 '이'라는 사실이 너무나 명백해서 0이란 값을 가집니다. 그런데 '디스플레이' 다음에는 어떤 값을 가질까요?

갑자기 값이 급증합니다. 그 이유는 문자 시퀀스 '디스플레이'라는 문자 시퀀스 다음에는 조사나 다른 단어가 나올 수 있는 다양한 경우가 있을 수 있기 때문입니다. 즉 하나의 단어가 끝나면 그 경계 부분부터 다시 브랜칭 엔트로피값이 증가함을 의미합니다. 그러면 이 값으로 단어를 판단하는 것이 가능하겠죠?

4. soynlp의 LTokenizer

띄어쓰기 단위로 잘 나뉜 문장은 L 토크나이저(LTokenizer)를 사용하면 좋습니다

한국어는 띄어쓰기 단위로 나눈 어절 토큰이 주로 L 토큰 + R 토큰의 형식을 가질 때가 많습니다. 예를 들어서 '공원에'는 '공원 + 에'로 나눌 수 있겠지요. 또는 '공부하는'은 '공부 + 하는'으로 나눌 수도 있을 것입니다. L 토크나이저는 L 토큰 + R 토큰으로 나누되, 점수가 가장 높은 L 토큰을 찾아내는 분리 기준을 가지고 있습니다.

In [ ]:
# LTokenizer

from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores = scores)
l_tokenizer.tokenize("국제사회와 우리의 노력들로 범죄를 척결하자", flatten=False)

[('국제사회', '와'), ('우리', '의'), ('노력', '들로'), ('범죄', '를'), ('척결', '하자')]

5. 최대 점수 토크나이저

최대 점수 토크나이저(MaxScoreTokenizer)는 띄어쓰기가 되어 있지 않은 문장에서 점수가 높은 글자 시퀀스를 순차적으로 찾아내는 토크나이저입니다. 띄어쓰기가 되어 있지 않은 문장을 넣어서 토큰화된 결과를 점수를 통해 알아보겠습니다.


In [ ]:
# MaxScoreTokenizer

from soynlp.tokenizer import MaxScoreTokenizer

maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']